In [15]:
import re
from tqdm import tqdm
import subprocess
import matplotlib.pyplot as plt
import numpy as np

In [16]:
import os

def list_files_in_directory(directory_path):
    # Check if the given directory exists
    if not os.path.isdir(directory_path):
        return "The specified directory does not exist."

    # List all entries in the directory
    files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
    return files

directory_path = "Benchmarks/combinational/rev_circuit"
files = list_files_in_directory(directory_path)
print(files)


['hwb10ps.qasm', 'hwb12_55998_198928.qasm', 'hwb9-1541.qasm', 'hwb4tc.qasm', 'gf2^12mult_177_753.qasm', 'hwb11-8214.qasm', 'gf2^127mult_16255_80771.qasm', 'gf2^10mult_109_509.qasm', 'mod5adders.qasm', 'nth_prime4_inc_15_51.qasm', 'hwb09-n9-gc4014-qc12293.qasm', 'hwb14ps.qasm', 'ham7-25-49.qasm', 'rd53-16-67.qasm', 'hwb07-n7-gc612-qc1737.qasm', 'nth_prime10_inc-n11-gc5207-qc50152.qasm', 'gf2^3mult_11_47.qasm', 'hwb6ps.qasm', 'hwb12-n12-gc42113-qc134334.qasm', 'ham15-70.qasm', 'hwb5-33-71.qasm', 'gf2^17mult_305_1461.qasm', 'hwb4_fc.qasm', 'hwb7-331-2609a.qasm', 'hwb11_32261_90745.qasm', 'mspk_nth_primes4_12.qasm', 'gf2^15mult_239_1139.qasm', 'hwb10-3631.qasm', 'nth_prime8_inc_3346_7618.qasm', 'hwb5-24-102.qasm', 'mspk_4_49_12.qasm', 'rd53d1-n7-gc12-qc95.qasm', 'gf2^18mult_375_1671.qasm', 'gf2^50_2647_12647.qasm', 'permanent3x3p3.qasm', 'rd53d2.qasm', 'hwb11-n11-gc21447-qc69153.qasm', 'hwb08-n8-gc1604-qc4862.qasm', 'mod5adder-15.qasm', 'gf2^7mult_55_251.qasm', 'ham15-109-214.qasm', 'hwb10

In [17]:
def read(file_path):
    # Regular expression patterns to match the lines of interest
    time_pattern = re.compile(r"time: ([\d\.]+)s")
    max_node_pattern = re.compile(r"MAX node: (\d+)")
    final_node_pattern = re.compile(r"final node: (\d+)")

    # Initialize lists to store the extracted information
    times = []
    max_nodes = []
    final_nodes = []

    # Read the file and extract information
    with open(file_path, 'r') as file:
        for line in file:
            # Check for time and add it to the times list
            time_match = time_pattern.search(line)
            if time_match:
                times.append(float(time_match.group(1)))
            
            # Check for MAX node and add it to the max_nodes list
            max_node_match = max_node_pattern.search(line)
            if max_node_match:
                max_nodes.append(int(max_node_match.group(1)))
            
            # Check for final node and add it to the final_nodes list
            final_node_match = final_node_pattern.search(line)
            if final_node_match:
                final_nodes.append(int(final_node_match.group(1)))
    return [times,max_nodes,final_nodes]


In [18]:
code = "LimTDD"
exper = "grover"
simulate = True
log_file_path = f"{code}_{exper}.txt"
error_log_path = f"{code}_{exper}_error.txt"

In [19]:
BenchmarkFolder = {"clifford":"Benchmarks/clifford/",
                   "clifford_cs":"Benchmarks/clifford_cs/",
                   "clifford_t":"Benchmarks/clifford_t/",
                   "bv":"Benchmarks/combinational/bv/",
                   "grover":"Benchmarks/combinational/grover/",
                   "pe":"Benchmarks/combinational/pe/",
                   "qft":"Benchmarks/qft/",
                   "qv":"Benchmarks/combinational/qv/",
                   "rev_circuit":"Benchmarks/rev_circuit/",
                   "vqe":"Benchmarks/vqe/",
                   }
BenchmarkFile = {"clifford":[f"{i}time.qasm" for i in range(1,1001)],
                 "clifford_cs":[f"{i}time.qasm" for i in range(1,1001)],
                 "clifford_t":[f"{i}time.qasm" for i in range(1,1001)],
                 "bv":[f"bv_{i}.qasm" for i in [30,50,100,200]],
                 "grover":[f"grover_{i}.qasm" for i in [9,19,29]],
                 "pe":[f"pe_{i}.qasm" for i in [10,20,30]],
                 "qft":[f"qft_{i}.qasm" for i in range(2,16)],
                 "qv": ['quantum_volume_n5_d5_i0.qasm', 'quantum_volume_n10_d5_i0.qasm', 'quantum_volume_n15_d5_i0.qasm',],
                 "rev_circuit":[f"rev_{i}.qasm" for i in range (1,3)],
            
                 "vqe":[f"vqe_{i}.qasm" for i in range(5,11)],
                }

In [20]:
test_cases = ["rev_circuit"]

In [21]:
for exper in test_cases:
    log_file_path = f"{code}_{exper}.txt"
    error_log_path = f"{code}_{exper}_error.txt"
    # Ensure the log file is empty at the start
    open(log_file_path, 'w').close()
    open(error_log_path, 'w').close()

    n = len(BenchmarkFile[exper])
    # Loop to run the command for each index, wrapped with tqdm for progress bar
    for i in tqdm(range(n), desc=f'Executing test about {exper} using {code}'):
        # Execute the C++ program and append its output to the log file
        if simulate:
            result = subprocess.run(["./build/test/test_data",BenchmarkFolder[exper]+BenchmarkFile[exper][i]], capture_output=True, text=True)
        else:
            result = subprocess.run(["./build/test/test_data",BenchmarkFolder[exper]+BenchmarkFile[exper][i], 1], capture_output=True, text=True)
        # Append the output to the log file
        with open(log_file_path, "a") as log_file:
            log_file.write(result.stdout)
        
        # Check if the command was successful
        if result.returncode != 0:
            with open(error_log_path, "a") as error_log:
                error_log.write(f"Program failed with argument {i}: {result.stderr}\n")

    print("All executions completed. Check", log_file_path, "for details.")

Executing test about rev_circuit using LimTDD:   0%|          | 0/2 [00:00<?, ?it/s]

Executing test about rev_circuit using LimTDD: 100%|██████████| 2/2 [00:00<00:00, 13.13it/s]

All executions completed. Check LimTDD_rev_circuit.txt for details.


In [15]:
for exper in test_cases:
    log_file_path = f"{code}_{exper}.txt"
    !cp {log_file_path} ../data/{exper}/{code}.txt

cp: cannot create regular file '../data/vqe/LimTDD.txt': No such file or directory


In [32]:
[times,maxNodes,_]=read(log_file_path)
# Example data
times = np.array(times)
memory_usage = np.array(maxNodes)  

# Calculate averages
avg_time = np.mean(times)
avg_memory = np.mean(memory_usage)

# Find indices of max and min execution times
max_time_idx = np.argmax(times)
min_time_idx = np.argmin(times)

# Create scatter plot
plt.scatter(times, memory_usage)

# Highlight max and min points
plt.scatter(times[max_time_idx], memory_usage[max_time_idx], color='red', label='Max Time', zorder=5)
plt.scatter(times[min_time_idx], memory_usage[min_time_idx], color='green', label='Min Time', zorder=5)

# Add average lines
plt.axhline(y=avg_memory, color='r', linestyle='--', label='Average MAX Node')
plt.axvline(x=avg_time, color='g', linestyle='--', label='Average Execution Time')

# Add annotations for max and min points
plt.annotate('Max Time', (times[max_time_idx], memory_usage[max_time_idx]), textcoords="offset points", xytext=(0,10), ha='center')
plt.annotate('Min Time', (times[min_time_idx], memory_usage[min_time_idx]), textcoords="offset points", xytext=(0,-15), ha='center')

plt.title(f'{code} Experiment Resource Usage')
plt.xlabel('Execution Time (s)')
plt.ylabel('MAX Nodes')
plt.legend()
plt.show()

ValueError: attempt to get argmax of an empty sequence